In [1]:
#!pip install -q langchain langchain-openai python-dotenv

In [4]:
from dotenv import load_dotenv
import os

load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")


if OPENAI_API_KEY.startswith("gsk_"):
    os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"
    MODEL_NAME = "llama-3.1-8b-instant"     


print("KEY HEAD:", OPENAI_API_KEY[:4])
print("MODEL  :", MODEL_NAME)


KEY HEAD: gsk_
MODEL  : llama-3.1-8b-instant


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0.3,           # 추천 일관성 ↑
    api_key=OPENAI_API_KEY,
)

stage1_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 세계 여행 도우미야. 사용자가 말한 여행지(도시/국가)를 보고, "
     "그곳에서 꼭 가볼 만한 대표 **관광 명소 하나**만 골라."),
    ("user",
     "여행지: {place}\n\n규칙: 추가 설명 없이 **명소 이름만** 한 줄로 출력해.")
])

stage1_chain = stage1_prompt | llm | StrOutputParser()


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0.3,          
    api_key=OPENAI_API_KEY,
)

stage1_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 세계 여행 도우미야. 사용자가 말한 여행지(도시/국가)를 보고, "
     "그곳에서 꼭 가볼 만한 대표 **관광 명소 하나**만 골라."),
    ("user",
     "여행지: {place}\n\n규칙: 추가 설명 없이 **명소 이름만** 한 줄로 출력해.")
])

stage1_chain = stage1_prompt | llm | StrOutputParser()

In [7]:
stage2_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 여행 가이드야. 주어진 명소에 대해 정확하고 간결하게 설명해."),
    ("user",
     "여행지: {place}\n"
     "명소: {landmark}\n\n"
     "아래 형식 그대로 한국어로 작성해줘.\n\n"
     "명소: {landmark}\n"
     "역사:\n"
     "1. ...\n2. ...\n3. ...\n"
     "특징:\n"
     "1. ...\n2. ...\n3. ...\n"
     "방문 팁:\n"
     "1. ...\n2. ...\n3. ...\n")
])

stage2_chain = stage2_prompt | llm | StrOutputParser()


In [8]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


linked_chain = (
    {"place": RunnablePassthrough()}           
    | {
        "landmark": stage1_chain,
        "place": RunnablePassthrough(),         
      }
    | stage2_chain
)

def run_travel_info(place: str):
    
    landmark = stage1_chain.invoke({"place": place}).strip()
    print(f"[1단계 결과] 추천 명소: {landmark}\n")

    details = stage2_chain.invoke({"place": place, "landmark": landmark})
    print("[2단계 결과]")
    print(details)

run_travel_info("로마")


[1단계 결과] 추천 명소: 콜로세움

[2단계 결과]
명소: 콜로세움

역사:
1. 콜로세움은 로마 제국의 황제 베스파시아누스와 그의 아들 티투스에 의해 72년부터 80년 사이에 지어졌습니다. 이 건물은 로마의 황제들이 개최하던 경기와 공연을 위해 지어졌으며, 로마 제국의 가장 대표적인 건축물 중 하나입니다.
2. 콜로세움은 로마의 황제들이 개최하던 경기와 공연을 위해 지어졌으며, 로마 제국의 가장 대표적인 건축물 중 하나입니다. 이 건물은 로마의 황제들이 개최하던 경기와 공연을 위해 지어졌으며, 로마 제국의 가장 대표적인 건축물 중 하나입니다.
3. 콜로세움은 로마의 황제들이 개최하던 경기와 공연을 위해 지어졌으며, 로마 제국의 가장 대표적인 건축물 중 하나입니다.

특징:
1. 콜로세움은 로마 제국의 가장 대표적인 건축물 중 하나로, 50,000명 이상의 관중을 수용할 수 있는 대형 경기장입니다.
2. 콜로세움은 로마 제국의 가장 대표적인 건축물 중 하나로, 50,000명 이상의 관중을 수용할 수 있는 대형 경기장입니다.
3. 콜로세움은 로마 제국의 가장 대표적인 건축물 중 하나로, 50,000명 이상의 관중을 수용할 수 있는 대형 경기장입니다.

방문 팁:
1. 콜로세움은 로마의 가장 대표적인 명소 중 하나로, 방문객이 많습니다. 따라서 방문하기 전에 시간을 예약하고, 관광객이 많은 시간을 피하는 것이 좋습니다.
2. 콜로세움은 로마의 가장 대표적인 명소 중 하나로, 방문객이 많습니다. 따라서 방문하기 전에 시간을 예약하고, 관광객이 많은 시간을 피하는 것이 좋습니다.
3. 콜로세움은 로마의 가장 대표적인 명소 중 하나로, 방문객이 많습니다. 따라서 방문하기 전에 시간을 예약하고, 관광객이 많은 시간을 피하는 것이 좋습니다.


In [9]:
for p in ["뉴욕", "도쿄", "서울"]:
    print("\n" + "="*60)
    run_travel_info(p)



[1단계 결과] 추천 명소: 타임스 스퀘어

[2단계 결과]
명소: 타임스 스퀘어

역사:
1. 1904년 뉴욕 타임즈가 본사를 두고 있는 곳으로 처음 이름이 붙여졌습니다.
2. 1920년대에 현재의 모습으로 재건되었습니다.
3. 1970년대부터 1980년대까지는 뉴욕의 중심부로 급성장하여 세계적인 명소로 자리잡았습니다.

특징:
1. 뉴욕 타임즈의 본사 건물이 위치하고 있습니다.
2. 타임스 스퀘어는 뉴욕의 중심부에 위치하여 다양한 관광 명소와 상점이 밀집되어 있습니다.
3. 타임스 스퀘어는 뉴욕의 대표적인 랜드마크 중 하나로 세계적인 명소로 유명합니다.

방문 팁:
1. 타임스 스퀘어는 뉴욕의 중심부에 위치하여 지하철역이 가까이 있습니다. 따라서 쉽게 접근할 수 있습니다.
2. 타임스 스퀘어는 다양한 관광 명소와 상점이 밀집되어 있습니다. 따라서 많은 시간을 투자하여 관광 명소와 상점을 둘러보는 것을 추천합니다.
3. 타임스 스퀘어는 뉴욕의 대표적인 랜드마크 중 하나로 사진을 찍기 좋은 장소입니다. 따라서 카메라를 준비하여 사진을 찍는 것을 추천합니다.

[1단계 결과] 추천 명소: 센토시지

[2단계 결과]
명소: 센토시지

역사:
1. 센토시지는 1964년 도쿄 올림픽을 위해 지어졌으며, 그 이후로 일본의 대표적인 공원으로 자리 잡고 있습니다.
2. 이 공원은 1969년 센토시지로 개명되었습니다.
3. 센토시지는 도쿄의 대표적인 관광 명소 중 하나로, 매년 수백만 명의 방문객이 찾고 있습니다.

특징:
1. 센토시지는 도쿄의 중심부에 위치해 있으며, 도쿄 타워와 센토시지의 아름다운 경관을 감상할 수 있습니다.
2. 센토시지는 다양한 식당과 상점이 있는 쇼핑 거리인 센토시지 구역을 가지고 있습니다.
3. 센토시지는 다양한 문화 행사와 이벤트를 개최하는 곳으로, 방문객들은 다양한 프로그램을 즐길 수 있습니다.

방문 팁:
1. 센토시지는 도쿄의 중심부에 위치해 있기 때문에, 지하철역이 많습니다. 따라서, 지하철을 이용하여 센토시지에 방문하는 것이 편리합니